In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
carvan_train = pd.read_csv("carvan_train.csv")

In [3]:
carvan_test = pd.read_csv("carvan_test.csv")

In [4]:
X = carvan_train.drop('V86',axis=1)
y = carvan_train['V86']

In [5]:
X = carvan_train.drop('V86',axis=1)
y = carvan_train['V86']

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

In [7]:
clf_gbm = GradientBoostingClassifier()

In [8]:
param_grid = {'learning_rate': np.linspace(0.1, 2, 150), 'min_samples_leaf': list(range(20, 65))}

In [9]:
from sklearn.model_selection import RandomizedSearchCV

In [10]:
randomsearch = RandomizedSearchCV(
    estimator = clf_gbm,
    param_distributions = param_grid,
    n_iter = 30,
    scoring='roc_auc', n_jobs=4, cv = 5, refit=True, return_train_score = True)

In [11]:
randomsearch.fit(X,y)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                        criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                    

In [12]:
cutoffs=np.linspace(0.01,0.99,99)

train_score=randomsearch.predict_proba(X)[:,1] 

real=y 

print(randomsearch.classes_)

[0 1]


In [13]:
FB_all=[]
for cutoff in cutoffs:

 predicted=(train_score>cutoff).astype(int)
 TP=((predicted==1) & (real==1)).sum()
 TN=((predicted==0) & (real==0)).sum()
 FP=((predicted==1) & (real==0)).sum()
 FN=((predicted==0) & (real==1)).sum()

 P=TP+FN
 N=TN+FP
 
 Sn=TP/P
 Sp=TN/N
 
 precision=TP/(TP+FP)
 recall=Sn

 KS=(TP/P)-(FP/N)

 FB=(5*precision*recall)/((4*precision)+recall)

 FB_all.append(FB)

In [14]:
max(FB_all)

0.5483870967741935

In [15]:
mycutoff=cutoffs[FB_all==max(FB_all)][0]
mycutoff

0.09999999999999999

In [16]:
test_score = randomsearch.predict_proba(carvan_test)[:,1]

In [17]:
test_classes=(test_score>mycutoff).astype(int)

In [18]:
test_score

array([0.01150524, 0.24217384, 0.088519  , ..., 0.09148241, 0.07938224,
       0.03910488])

In [19]:
test_classes

array([0, 1, 0, ..., 0, 0, 0])

In [20]:
pd.DataFrame(test_classes).to_csv("Ganesh_Patil_P2_part2.csv",index=False)